In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import warnings
warnings.simplefilter('ignore')

df = pd.read_csv("train.csv") 
df_test = pd.read_csv("test.csv") 


df = df[df["Country"]=="Japan"]
df = df[df["City"]=="Tokyo"]
df = df[df["month"]==1]

#df_test = df_test[df_test["Country"]=="Japan"]
#df_test = df_test[df_test["City"]=="Nagano"]

df['date'] = pd.to_datetime({'year': df['year'], 'month': df['month'], 'day': df['day']})
#df_test['date'] = pd.to_datetime({'year': df_test['year'], 'month': df_test['month'], 'day': df_test['day']})

df = df.drop(["date","id"], axis=1)
#df_test = df_test.drop(["date","id"], axis=1)

df_id = df.copy()
#df_test_id = df_test.copy()



df = df.fillna(0)
#df_test = df_test.fillna(0)

df

,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,...,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid
103,2019,1,1,Japan,Tokyo,35.6895,139.69171,1865,0.102,3.639,...,0.000,0.491,6.419,10.009,24,0.532,2.936,4.617,4.061,38.690
297,2019,1,2,Japan,Tokyo,35.6895,139.69171,45,0.090,0.122,...,0.500,1.155,3.156,1.642,75,9.748,13.201,13.726,3.692,45.542
492,2019,1,3,Japan,Tokyo,35.6895,139.69171,29,1.046,3.447,...,0.991,1.052,3.131,1.114,61,-14.871,17.738,20.843,11.323,16.473
689,2019,1,4,Japan,Tokyo,35.6895,139.69171,41,0.115,0.121,...,0.797,0.995,3.068,1.255,122,2.793,3.664,14.042,27.117,61.589
879,2019,1,5,Japan,Tokyo,35.6895,139.69171,2565,0.098,4.364,...,0.000,0.525,10.124,24.221,36,0.558,2.477,4.322,3.445,44.673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136939,2021,1,27,Japan,Tokyo,35.6895,139.69171,1737,0.096,4.266,...,0.208,1.947,21.834,70.898,759,-2.274,5.704,8.821,29.921,62.585
137108,2021,1,28,Japan,Tokyo,35.6895,139.69171,1430,0.098,4.621,...,0.096,1.938,20.883,97.865,880,-2.390,0.943,3.604,8.631,48.855
137270,2021,1,29,Japan,Tokyo,35.6895,139.69171,1527,0.099,3.240,...,0.102,2.569,19.829,85.211,770,-14.753,-8.438,4.515,90.240,30.563
137430,2021,1,30,Japan,Tokyo,35.6895,139.69171,1811,0.100,3.185,...,0.099,2.521,18.785,78.585,371,-12.066,-10.646,-6.104,8.599,20.512


In [9]:
le = LabelEncoder()

df["Country"] = le.fit_transform(df["Country"])
df["City"] = le.fit_transform(df["City"])

#df_test["Country"] = le.fit_transform(df_test["Country"])
#df_test["City"] = le.fit_transform(df_test["City"])

In [10]:
# 目的変数と説明変数の定義
X_data = df.drop(columns=['pm25_mid'])
y_data = df['pm25_mid']

# train_test_splitのインポート
from sklearn.model_selection import train_test_split

# shuffle=Falseとすることで時系列が混ざるのを防止

# 学習データおよび検証データと、評価データに80:20の割合で2分割する
X_trainval, X_test, y_trainval, y_test = train_test_split(X_data, y_data, test_size=0.20, shuffle=False)

# 学習データと検証データに75:25の割合で2分割する
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, shuffle=False)

print("finish")

finish


In [17]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm

# 関数get_standardized_tの定義
def get_standardized_t(X, num_date):
    X = np.array(X)
    X_t_list = []
    for i in tqdm(range(len(X) - num_date + 1)):
        X_t = X[i:i+num_date]
        scaler = StandardScaler()
        X_standardized = scaler.fit_transform(X_t)
        X_t_list.append(X_standardized)
    return np.array(X_t_list)

num_date = 10

# 学習用、検証用、評価用データの加工
X_train_t =  get_standardized_t(X_train, num_date=num_date)
X_val_t = get_standardized_t(X_val, num_date=num_date)
X_test_t = get_standardized_t(X_test, num_date=num_date)
#df_test = get_standardized_t(df_test, num_date=num_date)


print(X_train_t.shape)
print(X_val_t.shape)
print(X_test_t.shape)
#print(df_test.shape)

100%|██████████| 6/6 [00:00<00:00, 3008.11it/s]

(35, 10, 52)
(6, 10, 52)
(6, 10, 52)


In [18]:
# 目的変数の変形

y_train_t = y_train[num_date-1:]
y_val_t = y_val[num_date-1:]
y_test_t = y_test[num_date-1:]


In [19]:
#Sequentialのインポート
from keras.models import Sequential
# Dense、LSTMのインポート
from keras.layers import Dense, LSTM
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# ネットワークの各層のサイズの定義
num_l1 = 100
num_l2 = 20
num_output = 1

# 以下、ネットワークを構築
model = Sequential()
# 第1層
model.add(LSTM(units=num_l1,
                activation='relu',
                batch_input_shape=(None, X_train_t.shape[1], X_train_t.shape[2])))
# 第2層
model.add(Dense(num_l2, activation='relu'))
# 出力層
model.add(Dense(num_output, activation='linear'))
# ネットワークのコンパイル
opt = tf.keras.optimizers.Adam( learning_rate=0.01 )
#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.compile(loss=root_mean_squared_error, optimizer="Adam") #metrics=['accuracy'])

# モデルの学習の実行（学習の完了までには数秒から数十秒ほど時間がかかります。）
result = model.fit(x=X_train_t, y=y_train_t, epochs=200, batch_size=512, validation_data=(X_val_t, y_val_t))

# 評価データの正解率の計算
#loss, accuracy = model.evaluate( X_test_t, y_test_t)

# 評価データの正解率の表示
print()
#print('accuracy:', accuracy)

pred = model.predict(X_test_t)

print("RMSE:",np.sqrt(mean_squared_error(y_test_t, pred)))

Epoch 1/200
1/1 [==============================] - 1s 774ms/step - loss: 64.8673 - val_loss: 55.4895
Epoch 2/200
1/1 [==============================] - 0s 85ms/step - loss: 64.7806 - val_loss: 55.4308
Epoch 3/200
1/1 [==============================] - 0s 83ms/step - loss: 64.7214 - val_loss: 55.3634
Epoch 4/200
1/1 [==============================] - 0s 84ms/step - loss: 64.6614 - val_loss: 55.2861
Epoch 5/200
1/1 [==============================] - 0s 81ms/step - loss: 64.5905 - val_loss: 55.1939
Epoch 6/200
1/1 [==============================] - 0s 87ms/step - loss: 64.5039 - val_loss: 55.0880
Epoch 7/200
1/1 [==============================] - 0s 45ms/step - loss: 64.3954 - val_loss: 54.9649
Epoch 8/200
1/1 [==============================] - 0s 51ms/step - loss: 64.2670 - val_loss: 54.8159
Epoch 9/200
1/1 [==============================] - 0s 50ms/step - loss: 64.1131 - val_loss: 54.6332
Epoch 10/200
1/1 [==============================] - 0s 59ms/step - loss: 63.9258 - val_loss: 54.410

Epoch 81/200
1/1 [==============================] - 0s 47ms/step - loss: 6.2061 - val_loss: 19.6521
Epoch 82/200
1/1 [==============================] - 0s 47ms/step - loss: 6.0318 - val_loss: 20.1947
Epoch 83/200
1/1 [==============================] - 0s 53ms/step - loss: 5.8256 - val_loss: 19.4365
Epoch 84/200
1/1 [==============================] - 0s 50ms/step - loss: 5.5198 - val_loss: 19.5650
Epoch 85/200
1/1 [==============================] - 0s 48ms/step - loss: 5.2325 - val_loss: 19.4190
Epoch 86/200
1/1 [==============================] - 0s 47ms/step - loss: 4.9879 - val_loss: 18.8078
Epoch 87/200
1/1 [==============================] - 0s 47ms/step - loss: 4.8092 - val_loss: 19.5951
Epoch 88/200
1/1 [==============================] - 0s 50ms/step - loss: 4.9131 - val_loss: 17.8134
Epoch 89/200
1/1 [==============================] - 0s 48ms/step - loss: 5.6309 - val_loss: 20.4108
Epoch 90/200
1/1 [==============================] - 0s 49ms/step - loss: 6.1098 - val_loss: 18.7489


In [35]:
import itertools
pred = list(itertools.chain.from_iterable(pred))

pr = pd.Series(pred)
#pr.to_csv("LSTM_test.csv",index=None,header=None)

#　評価用データ
pr

0      26.232582
1      33.667614
2      89.377350
3      53.395348
4      29.828623
5      29.451986
6      67.990723
7      23.748053
8      44.290154
9      82.538666
10    113.324043
11     56.038330
12     78.172119
13     17.236017
14     79.059837
15     75.751709
16     30.362495
17     35.252018
18     40.953392
19     25.162157
20     83.631332
21     50.959915
22     32.574059
23     60.577663
24     49.739883
25     34.751492
26     35.157970
27     54.364586
28     61.710030
29     31.660290
30     45.037987
31     25.939230
32     92.459686
33     75.009064
34     52.956284
35     53.048920
36     66.986160
37     37.856003
38     28.575241
39     31.317472
40     41.098152
41     66.029007
42     33.449486
43     44.257828
44     70.509865
45     64.500610
46     22.936956
47     47.963875
48     70.049660
49     49.390869
50    105.314102
51     45.966873
52     35.303699
53     46.874298
54     52.648335
55    107.706848
56     60.162392
57     39.989342
58     38.1638

In [ ]:
import itertools
sample = pd.read_csv("submit_sample.csv",header=None) 

pred = model.predict(df_test)

pred = list(itertools.chain.from_iterable(pred))
pred_df = pd.Series(pred)

number = (num_date + n) - 1
num = np.zeros(number)
pred_num = pd.Series(num)

for i in range(number):
    pred_num[i] = pred_df[number+i]

prediction = pd.concat([pred_num, pred_df], axis=0).reset_index(drop=True)
prediction = pd.DataFrame(prediction, columns=["pred"])

prediction["id"] = df_test_id["id"]
prediction = prediction.sort_values(['id']).reset_index(drop=True)
prediction

sample[1] = prediction["pred"]
sample.to_csv("LSTM_sabmission.csv",index=None,header=None)
sample